In [1]:
import torch
try:
  import imageio
  import imageio_ffmpeg
except:
  !pip install imageio_ffmpeg
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
warnings.filterwarnings("ignore")

In [2]:
device = torch.device('cuda')
# There are four datasets available: vox, taichi, mgif, ted
dataset_name = 'vox'
source_image_path = input('Enter the path to the source image: ')
source_name = source_image_path.split('\\')[-1].split('.')[0]
new_background_path = input('Enter the path to the new background image: ')
driving_video_path = input('Enter the path to the video: ')
config_path = 'config/vox-256.yaml'
checkpoint_path = 'checkpoints/vox.pth.tar'
predict_mode = 'relative'
find_best_frame = True

# vox, ttaichi and mgif are in size 256 x 256, ted is in size 384 x 384
pixel = 256
if(dataset_name == 'ted'):
    pixel = 384

output_video_path = f'../outputs/{source_name}.mp4'

In [4]:
%cd Background-Substitution
from change_background import add_character_to_background

intermediate_path = f"../intermediate/{source_name}.jpg"
add_character_to_background(source_image_path, new_background_path, intermediate_path)
%cd ..

c:\Users\Sahil\Desktop\CV-Project\Background-Substitution
c:\Users\Sahil\Desktop\CV-Project


In [5]:
%cd Video-Generation
source_image = imageio.imread(intermediate_path)
reader = imageio.get_reader(driving_video_path)

source_image = resize(source_image, (pixel, pixel))[..., :3]

fps = reader.get_meta_data()['fps']
driving_video = []
try:
    for im in reader:
        driving_video.append(im)
except RuntimeError:
    pass
reader.close()

driving_video = [resize(frame, (pixel, pixel))[..., :3] for frame in driving_video]

def display(source, driving, generated=None):
    fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

    ims = []
    for i in range(len(driving)):
        cols = [source]
        cols.append(driving[i])
        if generated is not None:
            cols.append(generated[i])
        im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
        plt.axis('off')
        ims.append([im])

    ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
    plt.close()
    return ani


HTML(display(source_image, driving_video).to_html5_video())

c:\Users\Sahil\Desktop\CV-Project\Video-Generation


In [8]:
from demo import load_checkpoints
inpainting, kp_detector, dense_motion_network, avd_network = load_checkpoints(config_path = config_path, checkpoint_path = checkpoint_path, device = device)

from demo import make_animation
from skimage import img_as_ubyte

if predict_mode=='relative' and find_best_frame:
    from demo import find_best_frame as _find
    i = _find(source_image, driving_video, device.type=="cpu")
    print ("Best frame: " + str(i))
    driving_forward = driving_video[i:]
    driving_backward = driving_video[:(i+1)][::-1]
    predictions_forward = make_animation(source_image, driving_forward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions_backward = make_animation(source_image, driving_backward, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)
    predictions = predictions_backward[::-1] + predictions_forward[1:]
else:
    predictions = make_animation(source_image, driving_video, inpainting, kp_detector, dense_motion_network, avd_network, device = device, mode = predict_mode)

#save resulting video
imageio.mimsave(output_video_path, [img_as_ubyte(frame) for frame in predictions], fps=fps)

HTML(display(source_image, driving_video, predictions).to_html5_video())

169it [00:14, 11.34it/s]


Best frame: 52


100%|██████████| 53/53 [00:03<00:00, 15.59it/s]
